# Managing Azure Machine Learning Workspace with Managed Network

**Requirements** - In order to benefit from this tutorial, you will need:
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - reference the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create Azure Machine Learning workspace with Workspace Managed Network from Python SDK
- Get the details of a workspace from Python SDK
- Understand provisioning the Managed Network manually
- Manage the Isolation Mode of the Managed Network
- Configure User Defined Outbound Rules for the Managed Network

**Motivations** - The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. Ability to create and manage workspace is a prerequisite for any activity in Azure Machine Learning.

**Reference** - This example will act as a starting point to get going with the [Managed Network](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-managed-network) capabilities of AzureML workspaces. 

**Acronyms Used in this Notebook**
| Acronym | Meaning | Definition |
|-|-|-|
| AIO | AllowInternetOutbound | Isolation mode that allows all internet outbound traffic from the workspace managed network. |
| AOAO | AllowOnlyApprovedOutbound | The more restrictive isolation mode for the workspace managed network which restricts outbound traffic, except for a specified approved list of services, service tags, or FQDNs |
| PE | Private Endpoint | Azure Private Endpoint created to an ARM resource. A private endpoint is a network interface that uses a private IP address from the managed virtual network. This network interface connects you privately and securely to a service that's powered by Azure Private Link. |
| ServiceTag | Azure Service Tag | A service tag represents a group of IP address prefixes from a given Azure service. Microsoft manages the address prefixes encompassed by the service tag and automatically updates the service tag as addresses change, minimizing the complexity of frequent updates to network security rules. |
| FQDN | Fully Qualified Domain Name | The full domain name of an internet resource. |

# 1. Create a basic workspace

## 1.1. Import the required libraries

In [ ]:
# import required libraries for this notebook
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    Workspace,
    ManagedNetwork,
    IsolationMode,
    FqdnDestination,
    ServiceTagDestination,
    PrivateEndpointDestination,
)
from azure.identity import DefaultAzureCredential
import datetime

## 1.2 Configure where workspace needs to be created.
Before creating a workspace, we need identifier parameters - a subscription and resource group. We will use these parameters to define where the Azure Machine Learning workspace will be created.

The `MLClient` from `azure.ml` will be used to create the workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. reference the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Enter details of your subscription
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"

# get a handle to the subscription
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group)

## 1.3 Create the managed network settings with isolation mode AllowInternetOutbound (AIO)
In a workspace with AIO mode enabled, all internet outbound traffic from the managed VNet is allowed. To create the managed network for the workspace, we will define the following attributes:
- `isolation_mode` - Managed Network Isolation mode for the workspace
- `outbound_rules` - (Optional) The set of managed network outbound rules to allow outbound communication from the managed network.

Note: We can optionally add outbound rules when configuring the workspace (examples of rules shown below)
Note: PrivateEndpointOutboundRule is the only supported type of outbound rule for AIO isolation mode

In [ ]:
managed_network = ManagedNetwork(
    isolation_mode=IsolationMode.ALLOW_INTERNET_OUTBOUND, outbound_rules=[]
)

## 1.4 Create a basic workspace using the managed network
To create the workspace, we will define the following attributes
- `name` - Name of the workspace
- `managed_network` - The managed network settings configured for the workspace (from cell above)
- `location` - The Azure [location](https://azure.microsoft.com/en-us/global-infrastructure/services/?products=machine-learning-service) for workspace. For e.g. eastus, westus etc.

Using the `MLClient` created earlier, we will create the workspace. This command will start workspace creation and provide a confirmation.

In [ ]:
# Creating a unique workspace name with current datetime to avoid conflicts

mvnet_workspace_name = "mlw-mvnet-prod-" + datetime.datetime.now().strftime(
    "%Y%m%d%H%M"
)

ws_mvnet = Workspace(
    name=mvnet_workspace_name,
    location="eastus",
    managed_network=managed_network,
)

ws_mvnet = ml_client.workspaces.begin_create(ws_mvnet).result()
print(ws_mvnet)

## 1.5 Update the isolation mode to AllowOnlyApprovedOutbound (AOAO) 
The isolation mode for the workspace managed network can be changed to the more restrictive AOAO mode via workspace update. For a workspace with AOAO mode enabled, internet outbound traffic is blocked from the managed network, unless specified by an approved list of services, service tags, or FQDNs. After this update in isolation mode, we can see the difference in outbound rules of the managed network. Some additional rules are added to the network settings as `Required` outbound rules since communication outbound is removed so PEs are necessitated to certain workspace-associated resources and certain service tags are required for some system communication. We can also see that these required rules are `Inactive` and the managed network is also in `Inactive` state as managed network creation is deferred until a compute is created or the network is provisioned. Once the isolation mode is changed to AOAO, it can't not be reverted back to AIO or Disabled.

Using the `MLClient` created earlier, we will update the workspace. This command will start workspace update and provide a confirmation.

In [ ]:
ws_mvnet = ml_client.workspaces.get(mvnet_workspace_name)

ws_mvnet.managed_network.isolation_mode = IsolationMode.ALLOW_ONLY_APPROVED_OUTBOUND

aoao_ws = ml_client.workspaces.begin_update(ws_mvnet).result()

print(aoao_ws)

# 2. Provisioning the Network
Note: This step is optional for regular workspaces but is required for workspaces that will run Spark Training jobs. 

The managed network resources will be deferred on creation of the workspace until a compute is created or provision is called.

We can see that after provisioning the network, the status of the network and the outbound rules is `Active` which means that the rules are effective in the managed network (PEs created, NSG rules added for service tags, FQDN rules applied via managed firewall).

Instead of provisioning the network explictly, we could also create a compute in the workspace, to trigger the managed network creation and see the rules and network status become `Active`.

In [ ]:
ws_provisioned = ml_client.workspaces.begin_provision_network(
    workspace_name=mvnet_workspace_name
).result()

print(ws_provisioned)

# 3. Adding outbound rules to the workspace
Outbound rules can be added to the workspace's managed network to allow outbound communication from the network. The possible types are `ServiceTag`, `FQDN`, or `PrivateEndpoint`.

In this example, we will include all three types for demonstration purposes.

## 3.1 Add a ServiceTag outbound rule
To create a ServiceTag outbound rule, we will define the `ServiceTagDestination` Python object with the following attributes:
- `name` - Name of the ServiceTag outbound rule.
- `service_tag` - Service Tag of an Azure service that maps to predefined IP addresses for its service endpoints.
- `protocol` - Allowed transport protocol, can be "TCP", "UDP", "ICMP" or "*" for all supported protocols.
- `port_ranges` - A comma-separated list of single ports and/or range of ports, such as 80,1024-65535 that traffics are allowed.

To see available Azure Service tags, you can reference [Azure Virtual Network Service Tags documentation](https://learn.microsoft.com/en-us/azure/virtual-network/service-tags-overview#available-service-tags).

In [ ]:
ml_client.workspace_outbound_rules.begin_create(
    workspace_name=mvnet_workspace_name,
    rule=ServiceTagDestination(
        name="servicetagrule",
        service_tag="AzureCloud",
        protocol="TCP",
        port_ranges="80, 8080",
    ),
).result()

## 3.2 Add a FQDN outbound rule
Note: FQDN outbound rules are implemented using Azure Firewall. If you use outbound FQDN rules, charges for Azure Firewall are included in your billing.

To create a FQDN outbound rule, we will define the `FqdnDestination` Python object with the following attributes:
- `name` - Name of the FQDN outbound rule.
- `destination` - Fully qualified domain name to which outbound connections are allowed, for example: `"*.contoso.com"`. Multiple can also be used such as `"pytorch.org, *.pytorch.org"`

In [ ]:
ml_client.workspace_outbound_rules.begin_create(
    workspace_name=mvnet_workspace_name,
    rule=FqdnDestination(name="fqdnrule", destination="pypi.org"),
).result()

## 3.3 Add a PrivateEndpoint outbound rule
To create a PrivateEndpoint outbound rule, we will define the `PrivateEndpointDestination` Python object with the following attributes:
- `name` - Name of the ServiceTag outbound rule.
- `service_resource_id` - The resource URI (ARM ID) of the root service that supports creation of the private link.
- `subresource_target` - The subresource type for the service for the private link resource.
- `spark_enabled` - (Optional) Indicates if the private endpoint can be used for Spark jobs, default is “false”.

Refer to [AzureML Managed Network documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-managed-network#private-endpoints) for the currently supported Private Enpoint destinations. 

Refer to [Private Link documentation](https://learn.microsoft.com/en-us/azure/private-link/private-endpoint-overview#private-link-resource) for the possible sub resource targets for each resource provider.

Note: For simplicty of this demonstration, we will add a `table` Private Endpoint outbound rule to the workspace-associated storage account. (also note that file and blob are already default required rules on that storage account)

In [ ]:
ml_client.workspace_outbound_rules.begin_create(
    workspace_name=mvnet_workspace_name,
    rule=PrivateEndpointDestination(
        name="privateendpointrule",
        service_resource_id=ws_mvnet.storage_account,
        subresource_target="table",
    ),
).result()

## 3.4 Get the workspace object and check rules added
After the managed network was `Active`, the rules we added above were applied to the managed network of the workspace. In this case we should see all added user defined rules in `Active` as well.

In [ ]:
ws_mvnet = ml_client.workspaces.get(mvnet_workspace_name)

print(ws_mvnet)

# Clean up resources
Clean up the workspaces we created and its associated resources with `delete_dependent_resources`.

In [ ]:
ml_client.workspaces.begin_delete(
    name=mvnet_workspace_name, delete_dependent_resources=True
).result()